In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

#### Mergins all datasets together

In [0]:
def create_df(file):
  df = pd.read_csv(file)
  return df

In [0]:
df = create_df('english_5K_label_final.csv')


In [0]:
df.head()

,Unnamed: 0,Unnamed: 0.1,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,CAMEOCodeDescription,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_FullName,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,title,text,summary,summary_wrap,lab_final
0,0,0,25090,896079819,19200101,192001,1920,1920.0027,ELI,RETIRED,NaN,NaN,NaN,NaN,NaN,ELI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,192,Occupy territory,192,19,4,-9.5,4,1,4,-6.476684,4,"Pune, Maharashtra, India",IN,IN16,70178.0,18.5333,73.8667,-2108361,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Pune, Maharashtra, India",IN,IN16,70178.0,18.5333,73.8667,-2108361,20200101044500,https://punemirror.indiatimes.com/pune/cover-s...,pmc threatens to seize house of ex i t comm wh...,PMC threatens to seize house of ex I-T comm wh...,"In this light, he had also sent a legal notice...","In this light, he had also sent a legal notice...",Irrelevant
1,1,1,33280,896089642,19200101,192001,1920,1920.0027,COP,POLICEMAN,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,EDU,STUDENT,NaN,NaN,NaN,NaN,NaN,EDU,NaN,NaN,1,51,Praise or endorse,51,5,1,3.4,3,1,3,0.709220,4,"Mirzapur, Uttar Pradesh, India",IN,IN36,17924.0,27.2494,79.3139,10451923,4,"Mirzapur, Uttar Pradesh, India",IN,IN36,17924.0,27.2494,79.3139,10451923,4,"Mirzapur, Uttar Pradesh, India",IN,IN36,17924.0,27.2494,79.3139,10451923,20200101070000,https://www.mid-day.com/articles/mentalhood-ta...,mentalhood tandav the family man 2 breathe 2 s...,"\nMentalhood, Tandav, The Family Man 2, Breath...","Mirzapur 2Cast: Ali Fazal, Shweta Tripathi, Ra...","Mirzapur 2Cast: Ali Fazal, Shweta Tripathi, Ra...",Irrelevant
2,2,2,34788,896095544,19200101,192001,1920,1920.0027,COP,POLICE,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,CVL,CITIZEN,NaN,NaN,NaN,NaN,NaN,CVL,NaN,NaN,1,51,Praise or endorse,51,5,1,3.4,2,1,2,2.970297,4,"Nagpur, Maharashtra, India",IN,IN16,17784.0,21.1500,79.1000,-2105396,4,"Nagpur, Maharashtra, India",IN,IN16,17784.0,21.1500,79.1000,-2105396,4,"Nagpur, Maharashtra, India",IN,IN16,17784.0,21.1500,79.1000,-2105396,20200101081500,https://www.nagpurtoday.in/in-pic-nagpur-polic...,in pic nagpur police extend new years greeting...,In Pic: Nagpur Police extend New Year's greeti...,Nagpur: Amid cloudy day and chilling night the...,Nagpur: Amid cloudy day and chilling night the...,Irrelevant
3,3,3,35500,896092971,19200101,192001,1920,1920.0027,COP,POLICE,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,CRM,DEALER,NaN,NaN,NaN,NaN,NaN,CRM,NaN,NaN,1,10,"Make statement, not specified below",10,1,1,0.0,4,1,4,4.624277,4,"Pune, Maharashtra, India",IN,IN16,70178.0,18.5333,73.8667,-2108361,4,"Pune, Maharashtra, India",IN,IN16,70178.0,18.5333,73.8667,-2108361,4,"Pune, Maharashtra, India",IN,IN16,70178.0,18.5333,73.8667,-2108361,20200101074500,https://www.timesnownews.com/the-buzz/article/...,man asks for some drugs in exchange for dealer...,\n\n\nPune Police Twitter | Man asks for some ...,Pune police's witty response to man wins Twitt...,Pune police's witty response to man wins Twitt...,Irrelevant
4,4,4,47722,896113974,19200101,192001,1920,1920.0027,HLH,NURSE,NaN,NaN,NaN,NaN,NaN,HLH,NaN,NaN,IND,DELHI,IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,40,"Consult, not specified below",40,4,1,1.0,2,1,2,-4.068858,4,"Delhi, Delhi, India",IN,IN07,17911.0,

In [0]:
df['lab_final'].value_counts()

Irrelevant                            4207
Case_progress                          289
Harassment                             235
DV                                     233
Measures                               154
Verbal_Online_Harassment                52
MurderSuicide                           41
DVHarassment                             4
HarassmentMurderSuicide                  3
MurderSuicideHarassment                  2
Verbal_Online_HarassmentHarassment       2
DVMurderSuicide                          1
DVCase_progress                          1
MeasuresCase_progress                    1
HarassmentVerbal_Online_Harassment       1
HarassmentDV                             1
Name: lab_final, dtype: int64

In [0]:
df.loc[df['lab_final']=='DVMurderSuicide','lab_final']='DV'
df.loc[df['lab_final']=='DVCase_progress','lab_final']='DV'
df.loc[df['lab_final']=='MeasuresCase_progress','lab_final']='Measures'
df.loc[df['lab_final']=='Verbal_Online_HarassmentHarassment','lab_final']='Verbal_Online_Harassment'
df.loc[df['lab_final']=='HarassmentVerbal_Online_Harassment','lab_final']='Verbal_Online_Harassment'
df.loc[df['lab_final']=='DVHarassment','lab_final']='DV'
df.loc[df['lab_final']=='HarassmentDV','lab_final']='DV'
df.loc[df['lab_final']=='HarassmentMurderSuicide','lab_final']='MurderSuicide'
df.loc[df['lab_final']=='MurderSuicideHarassment','lab_final']='MurderSuicide'

In [5]:
df['lab_final'].value_counts()

Irrelevant                  4207
Case_progress                289
DV                           240
Harassment                   235
Measures                     155
Verbal_Online_Harassment      55
MurderSuicide                 46
Name: lab_final, dtype: int64

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
dummies = pd.get_dummies(df['lab_final'])
df = pd.concat([df, dummies], axis =1)

In [0]:
df['lab_final'].value_counts()

NON_D_VIOLENCE_ABOUT       3748
NO_VIOLENCE                1302
NON_D_VIOLENCE_DIRECTED     608
DV_OPINION_ADVOCATE         192
DV_OPINION_INFO_NEWS         94
DV_STORY                     58
DV_OPINION_NEWS_INFO         40
UNKNOWN                      24
DV_OPINION_DENIER            16
Name: lab_final, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
# 
df=df[['summary_wrap','lab_final']]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
   df['summary_wrap'], df['lab_final'], test_size=0.33, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
x_train_count = count_vectorizer.fit_transform(X_train)
x_train_count.shape

(3502, 20858)

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
x_train_tfidf = tfidf.fit_transform(x_train_count)
x_train_tfidf.shape


(3502, 20858)

In [0]:
from sklearn.pipeline import  Pipeline
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train_tfidf, y_train)
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

text_clf = text_clf.fit(X_train, y_train)

In [14]:
predictions = text_clf.predict(X_test)
predictions

array(['Irrelevant', 'Irrelevant', 'Irrelevant', ..., 'Irrelevant',
       'Irrelevant', 'Irrelevant'], dtype='<U24')

In [15]:
from sklearn.metrics import f1_score, accuracy_score
accuracy = accuracy_score(y_test,predictions)
print(accuracy)

f1 = f1_score(y_test,predictions,average='weighted')
print(f1)

0.8023188405797101
0.7148936051117899


In [16]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('cvec', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42))])
text_clf_svm = text_clf_svm.fit(X_train, y_train)
predictions_svm = text_clf_svm.predict(X_test)

f1_score_svm = f1_score(y_test, predictions_svm, average = 'weighted')
print(f'SVM score it is: {f1_score_svm}')

SVM score it is: 0.8445073195179927


In [17]:
sentence = 'The fact their might be so many marital rape and other sexual assaults within homes taking place right now with a greater rate, and no one wont be talking about it bcz CORONA.'
sample = text_clf_svm.predict([sentence])
sample

array(['Irrelevant'], dtype='<U24')

In [18]:
pip install -U imbalanced-learn

     |████████████████████████████████| 163kB 2.6MB/s 
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [19]:
X_train.shape, X_test.shape

((3502,), (1725,))

In [20]:
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.metrics import classification_report_imbalanced
from sklearn.feature_extraction.text import TfidfVectorizer
pipe = make_pipeline_imb(TfidfVectorizer(),
                         RandomOverSampler(),
                         MultinomialNB())

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

Using TensorFlow backend.


In [21]:
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

f1 = f1_score(y_test,y_pred,average='weighted')
print(f1)

0.8127536231884058
0.8366851874517364


In [22]:
print(classification_report_imbalanced(y_test, y_pred))

                                pre       rec       spe        f1       geo       iba       sup

           Case_progress       0.54      0.80      0.96      0.64      0.87      0.75        93
                      DV       0.44      0.80      0.95      0.57      0.87      0.75        79
              Harassment       0.35      0.73      0.94      0.48      0.83      0.67        79
              Irrelevant       1.00      0.84      0.99      0.91      0.91      0.81      1383
                Measures       0.47      0.63      0.97      0.54      0.78      0.59        59
           MurderSuicide       0.25      0.46      0.99      0.32      0.68      0.43        13
Verbal_Online_Harassment       0.47      0.37      1.00      0.41      0.61      0.34        19

             avg / total       0.89      0.81      0.98      0.84      0.89      0.78      1725



In [23]:
sentence = 'The fact their might be so many marital rape and other sexual assaults within homes taking place right now with a greater rate, and no one wont be talking about it bcz CORONA.'
sample = pipe.predict([sentence])
sample

array(['DV'], dtype='<U24')

In [103]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV


from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

import numpy as np
import matplotlib.pyplot as plt


#Preparing data
X = df['summary_wrap']
y = df['lab_final']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=42)
cv = CountVectorizer()
model_mult_nb = MultinomialNB()
alphas = np.linspace(0,2,20)[1:]

pipe1 = Pipeline([('cv',cv),
                ('model',model_mult_nb)
])
params = {'cv__stop_words': [['and','is',]]#,'civic','accord','camry','corolla','highlander']],
#           'cv__max_features':[4000,5000,6000,None],
#          'cv__ngram_range': [(1,1)],
#          'cv__min_df': [1,5,10,15],
#          'cv__max_df':[0.10,0.15,0.2,0.3],
#           'model__alpha' : alphas
    }
gs = GridSearchCV(pipe1, param_grid=params,cv=5)
print(gs.fit(X_train,y_train))
print(gs.best_params_)
print("Train Score: ", round(gs.best_score_,4))
print("Train Score: ", round(gs.score(X_test,y_test),4))


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('cv',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prepro

In [0]:
vectorizer = CountVectorizer(stop_words=['and','is'])
X_train_title_vec = vectorizer.fit_transform(X_train)
X_test_title_vec = vectorizer.transform(X_test)


In [72]:
print("Dic Size:", len(vectorizer.get_feature_names()))

Dic Size: 22218


In [74]:
cv = CountVectorizer()
model_mult_nb = MultinomialNB()
params = {'cv__stop_words': [['and','is']]}
gs = GridSearchCV(pipe1, param_grid=params,cv=5)
print(gs.fit(X_train,y_train))
print(gs.best_params_)
print("Train Score: ", round(gs.best_score_,4))
print("Train Score: ", round(gs.score(X_test,y_test),4))

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('cv',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prepro

In [115]:
vectorizer = CountVectorizer(stop_words=['and','is'])
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
rf = RandomForestClassifier(n_estimators=1000,random_state=42)
rf.fit(X_train_vec,y_train)
print("Cross Val Score: ",cross_val_score(rf,X_train_vec,y_train,cv=5).mean())
print("Train Score: ", round(rf.score(X_train_vec,y_train),4))
print("Train Score: ", round(rf.score(X_test_vec,y_test),4))

Cross Val Score:  0.8497448979591837
Train Score:  0.9995
Train Score:  0.8638


In [117]:
y_pred=rf.predict(X_test_vec)
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

f1 = f1_score(y_test,y_pred,average='weighted')
print(f1)
print(classification_report_imbalanced(y_test, y_pred))

0.8638102524866106
0.8291747596253548
                                pre       rec       spe        f1       geo       iba       sup

           Case_progress       0.88      0.60      1.00      0.71      0.77      0.57        72
                      DV       0.78      0.12      1.00      0.20      0.34      0.11        60
              Harassment       0.73      0.27      1.00      0.40      0.52      0.25        59
              Irrelevant       0.87      1.00      0.36      0.93      0.60      0.39      1052
                Measures       0.87      0.33      1.00      0.48      0.58      0.31        39
           MurderSuicide       0.00      0.00      1.00      0.00      0.00      0.00        11
Verbal_Online_Harassment       1.00      0.14      1.00      0.25      0.38      0.13        14

             avg / total       0.85      0.86      0.49      0.83      0.59      0.37      1307



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
vectorizer = CountVectorizer(stop_words=['and','is'])
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
et = ExtraTreesClassifier(random_state=42)
et.fit(X_train_vec,y_train)
print("Cross Val Score: ",cross_val_score(rf,X_train_vec,y_train,cv=5).mean())
print("Train Score: ", round(et.score(X_train_vec,y_train),4))
print("Test Score: ", round(et.score(X_test_vec,y_test),4))

Cross Val Score:  0.851530612244898
Train Score:  0.9995
Test Score:  0.8676


In [78]:
vectorizer = CountVectorizer(stop_words=['and','is'])
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

bagged = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42)
                            ,max_samples=0.5, max_features=0.5)
bagged.fit(X_train_vec,y_train)
print("Cross Val Score: ",cross_val_score(bagged,X_train_vec,y_train,cv=5).mean())
print("Train Score: ",bagged.score(X_train_vec,y_train))
print("Test Score: ",bagged.score(X_test_vec,y_test))

Cross Val Score:  0.8650510204081632
Train Score:  0.9543367346938776
Test Score:  0.8867635807192042


In [83]:
predictions = rf.predict(X_test_vec)
predictions

array(['Irrelevant', 'Irrelevant', 'Irrelevant', 'Irrelevant', ..., 'Irrelevant', 'Irrelevant', 'Irrelevant',
       'Irrelevant'], dtype=object)

In [84]:
from sklearn.metrics import f1_score, accuracy_score
accuracy = accuracy_score(y_test,predictions)
print(accuracy)

f1 = f1_score(y_test,predictions,average='weighted')
print(f1)

0.846212700841622
0.7989464741647327


In [86]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

                          precision    recall  f1-score   support

           Case_progress       0.88      0.50      0.64        72
                      DV       0.60      0.05      0.09        60
              Harassment       0.78      0.12      0.21        59
              Irrelevant       0.85      1.00      0.91      1052
                Measures       1.00      0.23      0.38        39
           MurderSuicide       0.00      0.00      0.00        11
Verbal_Online_Harassment       1.00      0.14      0.25        14

                accuracy                           0.85      1307
               macro avg       0.73      0.29      0.35      1307
            weighted avg       0.83      0.85      0.80      1307



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [88]:
predictions = bagged.predict(X_test_vec)
accuracy = accuracy_score(y_test,predictions)
print(accuracy)

f1 = f1_score(y_test,predictions,average='weighted')
print(f1)

print(classification_report(y_test, predictions))

0.8867635807192042
0.8709787432380971
                          precision    recall  f1-score   support

           Case_progress       0.74      0.62      0.68        72
                      DV       0.67      0.37      0.47        60
              Harassment       0.54      0.53      0.53        59
              Irrelevant       0.92      0.99      0.96      1052
                Measures       0.63      0.44      0.52        39
           MurderSuicide       0.00      0.00      0.00        11
Verbal_Online_Harassment       1.00      0.14      0.25        14

                accuracy                           0.89      1307
               macro avg       0.64      0.44      0.49      1307
            weighted avg       0.87      0.89      0.87      1307



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
predictions = et.predict(X_test_vec)
accuracy = accuracy_score(y_test,predictions)
print(accuracy)

f1 = f1_score(y_test,predictions,average='weighted')
print(f1)

print(classification_report(y_test, predictions))

0.8676358071920428
0.8384029410560432
                          precision    recall  f1-score   support

           Case_progress       0.90      0.60      0.72        72
                      DV       0.79      0.18      0.30        60
              Harassment       0.75      0.31      0.43        59
              Irrelevant       0.87      0.99      0.93      1052
                Measures       0.82      0.36      0.50        39
           MurderSuicide       0.00      0.00      0.00        11
Verbal_Online_Harassment       1.00      0.21      0.35        14

                accuracy                           0.87      1307
               macro avg       0.73      0.38      0.46      1307
            weighted avg       0.86      0.87      0.84      1307



In [0]:
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.metrics import classification_report_imbalanced
from sklearn.feature_extraction.text import TfidfVectorizer
pipe = make_pipeline_imb(TfidfVectorizer(),
                         RandomOverSampler(),
                         BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42)
                            ,max_samples=0.5, max_features=0.5))

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [99]:
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

f1 = f1_score(y_test,y_pred,average='weighted')
print(f1)

0.8806426931905126
0.879874590003058


In [100]:
print(classification_report_imbalanced(y_test, y_pred))

                                pre       rec       spe        f1       geo       iba       sup

           Case_progress       0.75      0.61      0.99      0.67      0.78      0.58        72
                      DV       0.59      0.70      0.98      0.64      0.83      0.66        60
              Harassment       0.49      0.66      0.97      0.57      0.80      0.62        59
              Irrelevant       0.95      0.95      0.80      0.95      0.87      0.77      1052
                Measures       0.57      0.64      0.99      0.60      0.79      0.61        39
           MurderSuicide       0.33      0.09      1.00      0.14      0.30      0.08        11
Verbal_Online_Harassment       0.80      0.29      1.00      0.42      0.53      0.27        14

             avg / total       0.89      0.88      0.84      0.88      0.85      0.73      1307



In [101]:
sentence = 'The fact their might be so many marital rape and other sexual assaults within homes taking place right now with a greater rate, and no one wont be talking about it bcz CORONA.'
sample = pipe.predict([sentence])
sample

array(['Measures'], dtype=object)

In [0]:
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.metrics import classification_report_imbalanced
from sklearn.feature_extraction.text import TfidfVectorizer
pipe = make_pipeline_imb(TfidfVectorizer(),
                         RandomOverSampler(),
                         RandomForestClassifier(random_state=42))

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [106]:
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

f1 = f1_score(y_test,y_pred,average='weighted')
print(f1)
print(classification_report_imbalanced(y_test, y_pred))

0.8745218056618209
0.8482355027857358
                                pre       rec       spe        f1       geo       iba       sup

           Case_progress       0.90      0.64      1.00      0.75      0.80      0.61        72
                      DV       0.78      0.23      1.00      0.36      0.48      0.21        60
              Harassment       0.70      0.36      0.99      0.47      0.59      0.33        59
              Irrelevant       0.88      0.99      0.44      0.93      0.66      0.46      1052
                Measures       0.88      0.36      1.00      0.51      0.60      0.34        39
           MurderSuicide       0.00      0.00      1.00      0.00      0.00      0.00        11
Verbal_Online_Harassment       1.00      0.14      1.00      0.25      0.38      0.13        14

             avg / total       0.86      0.87      0.55      0.85      0.65      0.44      1307



In [107]:
sentence = 'The fact their might be so many marital rape and other sexual assaults within homes taking place right now with a greater rate, and no one wont be talking about it bcz CORONA.'
sample = pipe.predict([sentence])
sample

array(['Irrelevant'], dtype=object)

In [0]:
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.metrics import classification_report_imbalanced
from sklearn.feature_extraction.text import TfidfVectorizer
pipe = make_pipeline_imb(TfidfVectorizer(),
                         RandomOverSampler(),
                         ExtraTreesClassifier(random_state=42))

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)


In [109]:
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

f1 = f1_score(y_test,y_pred,average='weighted')
print(f1)
print(classification_report_imbalanced(y_test, y_pred))

0.8607498087222647
0.8266287901022968
                                pre       rec       spe        f1       geo       iba       sup

           Case_progress       0.93      0.54      1.00      0.68      0.74      0.52        72
                      DV       0.73      0.18      1.00      0.29      0.43      0.17        60
              Harassment       0.67      0.17      1.00      0.27      0.41      0.15        59
              Irrelevant       0.86      1.00      0.35      0.92      0.59      0.37      1052
                Measures       0.84      0.41      1.00      0.55      0.64      0.39        39
           MurderSuicide       0.00      0.00      1.00      0.00      0.00      0.00        11
Verbal_Online_Harassment       1.00      0.14      1.00      0.25      0.38      0.13        14

             avg / total       0.85      0.86      0.48      0.83      0.58      0.35      1307

